# Active Seasons

Prints seasons that a housewife was active in a main role.

## Import Libraries

In [9]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
url_list = pd.read_csv('./data/series_urls.csv')

## Scrape Table Data and Print

In [ ]:
# Loop through the list of URLs for each series.
for index, row in url_list.iterrows():
    series = row['Series']
    soup = BeautifulSoup(urlopen(row['URL']))
    table = soup.find('table', class_='wikitable')
    rows = table.find_all('tr')

    # Loop through rows containing a housewife's name and their active seasons.
    for row in rows[2:]:
        # Ignore Friends of the Housewives since they're not the main cast.
        if row.text.strip().lower() == 'friends of the housewives':
            break

        tds = row.find_all('td')
        name = tds[0].text.strip()
        name_parts = name.split(' ', 1)
        first_name = name_parts[0].replace("'", "''")
        last_name = name_parts[1].replace("'", "''")

        # Elizabeth Lyn Vargas is the only housewife whose credited middle name is not part of a double surname.
        if last_name == 'Lyn Vargas':
            first_name = 'Elizabeth Lyn'
            last_name = 'Vargas'

        count = 1

        for td in tds[1:]:
            role = td.text.strip()

            # Get the number of columns that the role spans.
            columns = td.get('colspan')

            # Remove non-number characters.
            columns = re.findall(r'\d+', columns)[0]
            columns = int(columns)

            # Ignore seasons where a housewife did not appear in a main role but
            # add to count so we know what season we are on if they do later.
            if role == '' or role == 'Guest' or role == 'Friend':
                count += columns
                continue
            
            for i in range(columns):
                print(f"""(
    (
        SELECT
            season_id
        FROM
            seasons
        WHERE
            series_id = '{series}'
            AND season_number = {count + i}
    ),
    (
        SELECT
            housewife_id
        FROM
            housewives
        WHERE
            first_name = '{first_name}'
            AND last_name = '{last_name}'
    )
),""")
            count += columns

            # print(f"Active Seasons: {count - 1}")